# OpenAI o4-mini モデル テキスト生成ノートブック（Google Colab版）

このノートブックでは、**Google Colaboratory**でOpenAI の o4-mini モデルを使用してテキスト生成を行います。

## 必要な準備
1. **OpenAI API キーの取得**
2. **Google Colaboratoryでの実行**（ローカル環境は不要）
3. **APIキーの設定**（Google Colab専用の方法を使用）

## Google Colabでの環境変数設定方法

### 🔒 **推奨方法: Google Colab Secrets機能**
1. 左サイドバーの「🔑」アイコン（Secrets）をクリック
2. 「Add new secret」をクリック
3. Name: `OPENAI_API_KEY`
4. Value: あなたのOpenAI APIキー
5. 「Notebook access」をオンにする

### 📝 **代替方法: 直接入力**
セルでAPIキーを直接入力（**実行後は必ずセルを削除してください**）

### ⚠️ **重要な注意事項**
- **APIキーは機密情報です**
- **ノートブックを共有する前にAPIキーを含むセルは削除してください**
- **Google Colab Secretsの使用を強く推奨します**

## 実行環境
- ✅ **Google Colaboratory**で動作確認済み
- ✅ **追加インストール不要**（必要なライブラリは自動インストール）
- ✅ **無料版で利用可能**

## 1. 必要なライブラリのインストールとインポート

Google Colabでは、openaiライブラリを最初にインストールする必要があります。

In [ ]:
# Google Colabで必要なライブラリをインストール
!pip install openai

# 必要なライブラリをインポート
import os
import openai
from google.colab import userdata  # Google Colab専用のライブラリ

print("✅ ライブラリのインストールとインポートが完了しました。")

## 2. Google ColabでのOpenAI API キー設定

Google Colabには専用のSecrets機能があります。これが最も安全な方法です。

In [ ]:
# Google Colab SecretsからAPIキーを取得（推奨方法）
try:
    # Google Colab Secretsから取得
    oai_key = userdata.get('OPENAI_API_KEY')
    openai.api_key = oai_key
    print("✅ Google Colab SecretsからAPIキーを取得しました")
    print("🔒 セキュアな方法でAPIキーが設定されました")
    
except Exception as e:
    print("⚠️ Google Colab Secretsからの取得に失敗しました")
    print("以下の手順でSecretsを設定してください:")
    print("1. 左サイドバーの🔑アイコンをクリック")
    print("2. 'Add new secret'をクリック")
    print("3. Name: OPENAI_API_KEY")
    print("4. Value: あなたのAPIキー")
    print("5. 'Notebook access'をオンにする")
    print("\\n代替方法として、次のセルで直接入力できます（非推奨）")

# 【非推奨】代替方法: 直接入力
# ⚠️ 実行後は必ずこのセルを削除してください ⚠️
# ⚠️ ノートブックを共有する前に確実に削除してください ⚠️

# 以下のコメントアウトを外してAPIキーを設定（Secretsが使えない場合のみ）
# openai.api_key = "sk-proj-your-actual-api-key-here"
# print("⚠️ APIキーを直接設定しました - セキュリティのためこのセルを削除してください"

# より安全な方法：getpassを使用した入力
from getpass import getpass

print("🔐 APIキーを入力してください（入力時は表示されません）:")
manual_key = getpass("OpenAI API Key: ")

if manual_key.strip():
    openai.api_key = manual_key
    print("✅ APIキーが設定されました")
    print("🔒 入力されたキーは画面に表示されませんでした")
else:
    print("❌ APIキーが入力されませんでした")

In [ ]:
def generate_text(prompt: str, show_debug: bool = False) -> str:
    """
    ChatCompletion API を用いて o4-mini モデルから応答を取得する関数。

    Args:
        prompt (str): ユーザーからのプロンプト文字列
        show_debug (bool): デバッグ情報を表示するかどうか（デフォルト: False）
    
    Returns:
        str: モデルからの生成テキスト
    """
    try:
        # OpenAI APIを呼び出してテキストを生成
        response = openai.chat.completions.create(
            model="o4-mini",  # 使用するモデル名
            messages=[
                {
                    "role": "system", 
                    "content": (
                        "あなたは親切で知識豊富なアシスタントです。ユーザーの質問に対して明確で分かりやすい回答を提供してください。"
                        "常に具体的な内容を含む回答をし、空の応答は絶対に返さないでください。"
                    )
                },
                {"role": "user", "content": prompt}  # ユーザーからの質問
            ],
            max_completion_tokens=1000,  # 生成する最大トークン数
        )
        
        # デバッグ情報を表示（オプション）
        if show_debug:
            print("=== デバッグ情報 ===")
            print(f"モデル: {response.model}")
            print(f"使用トークン数: {response.usage.total_tokens}")
            print(f"完了理由: {response.choices[0].finish_reason}")
            print("==================")
        
        # 生成されたテキストを取得
        content = response.choices[0].message.content
        
        # 空の応答の場合はエラーメッセージを返す
        if not content:
            return "応答が生成されませんでした。再度お試しください。"
        
        return content.strip()
    
    except Exception as e:
        # エラーが発生した場合の処理
        return f"エラーが発生しました: {str(e)}"

print("✅ テキスト生成関数が定義されました")

## 4. 基本的な使用例

まずは簡単な質問でテキスト生成を試してみましょう。

In [ ]:
# 基本的な使用例
simple_prompt = "こんにちは！調子はどうですか？"

print("=" * 60)
print("基本的な使用例")
print("=" * 60)
print(f"プロンプト: {simple_prompt}")
print("-" * 60)
print("AI応答:")
print("-" * 60)

# テキスト生成を実行
result = generate_text(simple_prompt)
print(result)
print("=" * 60)

## 5. 技術的な質問の例

より複雑な技術的な質問を試してみましょう。

In [ ]:
# 技術的な質問の例
tech_prompt = "LLMの最新の技術動向について簡潔に教えてください。"

print("=" * 60)
print("技術的な質問の例")
print("=" * 60)
print(f"プロンプト: {tech_prompt}")
print("-" * 60)
print("AI応答:")
print("-" * 60)

# デバッグ情報も表示してテキスト生成を実行
result = generate_text(tech_prompt, show_debug=True)
print(result)
print("=" * 60)

## 6. インタラクティブな質問

自由に質問を入力してAIと対話できるセルです。

In [ ]:
# ここに自由に質問を入力してください
your_prompt = "Pythonでファイルを読み込む方法を教えてください。"

print("=" * 60)
print("あなたの質問")
print("=" * 60)
print(f"プロンプト: {your_prompt}")
print("-" * 60)
print("AI応答:")
print("-" * 60)

# テキスト生成を実行
result = generate_text(your_prompt)
print(result)
print("=" * 60)

## 7. 複数の質問を一度に処理

複数の質問をリストにして、一度に処理する例です。

In [ ]:
## 8. Google Colabでの使用上の注意とトラブルシューティング

### Google Colab特有の注意点

1. **セッション切断**: 長時間使用しないとセッションが切断されます
2. **ランタイム制限**: 無料版では連続使用時間に制限があります
3. **APIキー管理**: Secretsを使用して安全に管理してください

### よくあるエラーと対処法

1. **APIキーエラー**: Google Colab Secretsが正しく設定されているか確認
2. **レート制限エラー**: APIの使用制限に達した場合は時間をおいて再実行
3. **ライブラリエラー**: セッション再起動後は最初のセルから再実行

### パフォーマンスのヒント

- GPU/TPUは不要（テキスト生成のみの場合）
- `max_completion_tokens` を調整してトークン使用量を制御
- プロンプトを明確で具体的にする

# Google Colab環境情報の確認
import sys

print("=== Google Colab 環境情報 ===")
print(f"Python バージョン: {sys.version}")
print(f"実行環境: Google Colaboratory")
print(f"OpenAI ライブラリ: インストール済み")

# APIキー設定状況の確認（実際のキーは表示しない）
if hasattr(openai, 'api_key') and openai.api_key:
    print(f"API キー設定: ✅ 設定済み (長さ: {len(openai.api_key)} 文字)")
else:
    print("API キー設定: ❌ 未設定")

print("========================")
print("🚀 Google Colabでの実行準備が完了しました！")
print("💡 上記のセルを順番に実行してAIとの対話を開始してください")

In [ ]:
# システム情報の確認
print("=== システム情報 ===")
print(f"Python バージョン: {os.sys.version}")
print(f"OpenAI ライブラリ: インポート済み")
print(f"API キー設定: {'✅ 設定済み' if os.getenv('OPENAI_API_KEY') else '❌ 未設定'}")
print("===================")